In [36]:
import requests
from bs4 import BeautifulSoup
import random
import json


user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:85.0) Gecko/20100101 Firefox/85.0",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Mobile/15E148 Safari/604.1"
]

headers = {
    'User-Agent': random.choice(user_agents),
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.9'
}

# Start a session
session = requests.Session()
session.headers.update(headers)

login_url = 'https://mls.foreclosure.com/login.html'

login_data = {
    'username': 'haiwen.zhang@gmail.com',  
    'password': 'MRGSY_772'   
}

# Send POST request to login
# response = session.post(login_url, data=login_data, headers=headers)
response = session.post(login_url, data=login_data)


# Check if login was successful
if response.ok:
    print("Login successful")
#     session.headers.update({'Referer': 'https://mls.foreclosure.com/'})
    
    data_url = 'https://mls.foreclosure.com/listing/search.html?ci=austin&st=tx'
    response_next = session.get(data_url, headers=headers)
    soup = BeautifulSoup(response_next.text, 'html.parser')
    scripts = soup.find_all('script', type='application/ld+json')
    # This will store each property's basic details
    properties = []
    # Extract basic details from JSON data
    for script in scripts:
        try:
            data = json.loads(script.string)
            properties.append({
                'street_address': data.get('streetAddress'),
                'locality': data.get('addressLocality'),
                'region': data.get('addressRegion'),
                'postal_code': data.get('postalCode'),
                'latitude': data.get('geo', {}).get('latitude'),
                'longitude': data.get('geo', {}).get('longitude'),
                'details_url': data.get('url'),
            })
        except json.JSONDecodeError:
            continue
    print(properties)
else:
    print(response)
    print("Login failed")
# Close the session when done
session.close()


Login successful
[{'street_address': None, 'locality': None, 'region': None, 'postal_code': None, 'latitude': '30.371', 'longitude': '-97.767', 'details_url': 'https://mls.foreclosure.com/address/Harvestman-Cv-Austin-TX-78731/61865739_lid'}, {'street_address': None, 'locality': None, 'region': None, 'postal_code': None, 'latitude': '30.306', 'longitude': '-97.680', 'details_url': 'https://mls.foreclosure.com/address/Sweeney-Ln-Austin-TX-78723/61854517_lid'}, {'street_address': None, 'locality': None, 'region': None, 'postal_code': None, 'latitude': '30.255', 'longitude': '-97.712', 'details_url': 'https://mls.foreclosure.com/address/N-Pleasant-Valley-Rd-Austin-TX-78702/61854687_lid'}, {'street_address': None, 'locality': None, 'region': None, 'postal_code': None, 'latitude': '30.362', 'longitude': '-97.974', 'details_url': 'https://mls.foreclosure.com/address/White-Sands-Dr-Lakeway-TX-78734/61854810_lid'}, {'street_address': None, 'locality': None, 'region': None, 'postal_code': None, 

In [5]:
import requests
from bs4 import BeautifulSoup

In [121]:
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:85.0) Gecko/20100101 Firefox/85.0",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Mobile/15E148 Safari/604.1"
]

headers = {
    'User-Agent': random.choice(user_agents),
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.9'
}

def login_to_site():
    """Create a session and log in to the website."""
    login_url = 'https://mls.foreclosure.com/login.html'
    login_data = {
        'username': 'haiwen.zhang@gmail.com',
        'password': 'MRGSY_772'
    }
    session = requests.Session()
    response = session.post(login_url, data=login_data, headers=headers)
    
    if response.ok:
        print("Login successful")
        return session
    else:
        print("Login failed")
        return None

def extract_properties(soup):
    """Extracts basic property details from JSON embedded in script tags."""
    properties = []
    scripts = soup.find_all('script', type='application/ld+json')
    for script in scripts:
        try:
            data = json.loads(script.string)
            properties.append({
                'street_address': data.get('streetAddress'),
                'locality': data.get('addressLocality'),
                'region': data.get('addressRegion'),
                'postal_code': data.get('postalCode'),
                'latitude': data.get('geo', {}).get('latitude'),
                'longitude': data.get('geo', {}).get('longitude'),
                'details_url': data.get('url'),
            })
        except json.JSONDecodeError:
            continue
    return properties

def fetch_details(session, properties):
    """Fetch and parse detailed property information."""
    for property in properties:
        details_url = property['details_url']
        if not details_url.startswith('http'):
            details_url =  'http://' + details_url
        
        response = session.get(details_url, headers=headers, cookies=session.cookies)
        soup = BeautifulSoup(response.text, 'html.parser')
        property_info = {}
        print(soup)

        info_items = soup.find_all('li', class_='lci')
        for item in info_items:
            label_span = item.find('span', class_='attr-label')
            value_a = item.find('a')
            label = label_span.get_text(strip=True) if label_span else None
            value = value_a.get_text(strip=True) if value_a else 'Details require login'
            if label:
                property_info[label] = value

        property.update(property_info)

In [124]:
def main():
    search_url = 'https://mls.foreclosure.com/listing/search.html?ci=austin&st=tx'
    session = login_to_site()
    if session:
        response = session.get(search_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        properties = extract_properties(soup)
        print('properties successfully extracted', len(properties))
#         fetch_details(session, properties)
        
#         for property in properties:
#             if 'Day(s) On Site:' in property:
#                 for key, value in property.items():
#                     if value is not None:
#                         print(f'{key}: {value}')
        
        session.close() 

In [125]:
main()

Login successful
properties successfully extracted 25


In [62]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Configure Selenium WebDriver
options = Options()
options.headless = True  # Optional: Run in headless mode
driver = webdriver.Chrome(options=options)

try:
    # Open the login page
    driver.get("https://mls.foreclosure.com/login.html")

    # Wait for the login input fields to be present
    username_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "key"))
    )
    password_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "password"))
    )

    # Fill in the login credentials
    username_input.send_keys("haiwen.zhang@gmail.com")
    password_input.send_keys("MRGSY_772")

    # Identify and click the sign-in button
    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "btnLoginUsernamePassword"))
    )
    login_button.click()

    print('login successfully')
    
    # Navigate to the details page or perform other actions as required
    driver.get("https://mls.foreclosure.com/listing/search.html?ci=austin&st=tx")

    # Here you can add more processing logic as needed
    content = driver.page_source

finally:
    # Always close the driver
    driver.quit()


login successfully


In [150]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import random


def slenium_login_to_site():
    """Create a browser session and log in to the website."""
    try:
        driver.get("https://mls.foreclosure.com/login.html")
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "key"))).send_keys("haiwen.zhang@gmail.com")
        driver.find_element(By.NAME, "password").send_keys("MRGSY_772")
        driver.find_element(By.ID, "btnLoginUsernamePassword").click()
        print("Login successful")
        return True
    except Exception as e:
        print("Login failed:", e)
        return False

def slenium_extract_properties():
    """Extracts basic property details from JSON embedded in script tags."""
    properties = []
    scripts = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "//script[@type='application/ld+json']"))
    )
    #     print(scripts)
    for script in scripts:
        try:
            data = json.loads(script.get_attribute('innerHTML'))
            properties.append({
                'street_address': data.get('streetAddress'),
                'locality': data.get('addressLocality'),
                'region': data.get('addressRegion'),
                'postal_code': data.get('postalCode'),
                'latitude': data.get('geo', {}).get('latitude'),
                'longitude': data.get('geo', {}).get('longitude'),
                'details_url': data.get('url'),
            })
        except json.JSONDecodeError:
            print('error')
            continue
    return properties

def slenium_fetch_details(properties):
    """Fetch and parse detailed property information using Selenium."""
    counter = 0
    for property in properties:
        if counter > 20:
            break
        details_url = property['details_url']
        if not details_url.startswith('http'):
            details_url = 'https://' + details_url
        
        driver.get(details_url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'li.lci'))
        )
        
        info_items = driver.find_elements(By.CSS_SELECTOR, 'li.lci')
        property_info = {}
#         print(info_items)
        for item in info_items:
            try:
                label_span = item.find_element(By.CSS_SELECTOR, 'span.attr-label')
                value_span = item.find_element(By.CSS_SELECTOR, 'span.value')
                label = label_span.text.strip().rstrip(':') if label_span else None
                value = value_span.text.strip() if value_span else 'Details not available'
                if label:
                    property_info[label] = value
            except NoSuchElementException:
                print("Expected elements not found within the item, skipping...")
                continue
        print(property_info)
        property.update(property_info)
        counter += 1
        if counter % 5 == 0:
            print(f"Scraping property No.{counter}")
    return properties

In [151]:
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:85.0) Gecko/20100101 Firefox/85.0",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Mobile/15E148 Safari/604.1"
]

options = Options()
# options.add_argument(f"user-agent={random.choice(user_agents)}")
options.add_argument(f"user-agent={user_agents[0]}")

driver = webdriver.Chrome(options=options)

if slenium_login_to_site():
    driver.get("https://mls.foreclosure.com/listing/search.html?ci=austin&st=tx")
    properties = slenium_extract_properties()
    print('properties successfully extracted, total: ', len(properties))
    properties_updated = slenium_fetch_details(properties)
    print('properties details extracted')

#     for property in properties_updated:
#         print(property) 

    driver.quit()


Login successful
properties successfully extracted, total:  25
{'Bathrooms': '4.5', 'Living Area Size': '4,199', 'Year Built': '2002', 'Day(s) On Site': '1', 'Subdivision': 'CANYON RIM', 'Trustee Name': 'RANDOLPH N OSHEROW', 'Trustee Address': '342 W Woodlawn Ave Ste 100', 'Trustee City': 'San Antonio', 'Trustee State': 'TX', 'Trustee Zip': '78212', 'Trustee Phone': '(210) 738-3001', 'Court Name': 'District TXW, Division 1', 'Court Address': '903 San Jacinto Blvd, Suite 322', 'Baths': '4.5', 'Full Baths': '4', 'Half Baths': '1', 'No. of Stories': '2', 'Parking': 'ATTACHED GARAGE', 'Parking Sqft': '600', 'Air Conditioning': 'CENTRAL', 'Fireplaces': 'YES', 'Pool': 'CONCRETE', 'Construction Type': 'WOOD/BRICK', 'Foundation Type': 'SLAB', 'Heating Type': 'CENTRAL', 'Roof Material': 'COMPOSITION', 'Roof Type': 'HIP', 'Land Area Sqft': '10,781', 'APN': '495993', 'Property Type': 'Single Family Residence', 'Living Sqft': '4,199', 'Garage': 'ATTACHED', 'Attorney Law Firm': 'LAW OFFICE OF SUSAN

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x0000000100a568dc chromedriver + 4368604
1   chromedriver                        0x0000000100a4ed70 chromedriver + 4337008
2   chromedriver                        0x0000000100672c04 chromedriver + 289796
3   chromedriver                        0x00000001006b4e00 chromedriver + 560640
4   chromedriver                        0x00000001006ed5ec chromedriver + 792044
5   chromedriver                        0x00000001006a9ab4 chromedriver + 514740
6   chromedriver                        0x00000001006aa50c chromedriver + 517388
7   chromedriver                        0x0000000100a1adf8 chromedriver + 4124152
8   chromedriver                        0x0000000100a1fbe8 chromedriver + 4144104
9   chromedriver                        0x0000000100a007c0 chromedriver + 4016064
10  chromedriver                        0x0000000100a20518 chromedriver + 4146456
11  chromedriver                        0x00000001009f2274 chro

In [152]:
for property in properties_updated:
    for key, value in property.items():
        if value is not None:
            print(f'{key}: {value}')


latitude: 30.3710391
longitude: -97.7668617
details_url: https://mls.foreclosure.com/address/7801-Harvestman-Cv-Austin-TX-78731/61865739_lid
Bathrooms: 4.5
Living Area Size: 4,199
Year Built: 2002
Day(s) On Site: 1
Subdivision: CANYON RIM
Trustee Name: RANDOLPH N OSHEROW
Trustee Address: 342 W Woodlawn Ave Ste 100
Trustee City: San Antonio
Trustee State: TX
Trustee Zip: 78212
Trustee Phone: (210) 738-3001
Court Name: District TXW, Division 1
Court Address: 903 San Jacinto Blvd, Suite 322
Baths: 4.5
Full Baths: 4
Half Baths: 1
No. of Stories: 2
Parking: ATTACHED GARAGE
Parking Sqft: 600
Air Conditioning: CENTRAL
Fireplaces: YES
Pool: CONCRETE
Construction Type: WOOD/BRICK
Foundation Type: SLAB
Heating Type: CENTRAL
Roof Material: COMPOSITION
Roof Type: HIP
Land Area Sqft: 10,781
APN: 495993
Property Type: Single Family Residence
Living Sqft: 4,199
Garage: ATTACHED
Attorney Law Firm: LAW OFFICE OF SUSAN G TAYLOR
Attorney Name: SUSAN G. TAYLOR
Attorney Address: 1777 NE Loop 410 # 610
Atto

In [117]:
from bs4 import BeautifulSoup
import requests

def extract_properties():
    search_url = 'https://mls.foreclosure.com/listing/search.html?ci=austin&st=tx'

    # Send an HTTP GET request to the URL
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(search_url, headers=headers)

    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'html.parser')
    # Initialize a list to store property details
    properties = []

    # Find all property listings
    listings = soup.find_all('div', class_='listingRow')
#     print(listings)
    for listing in listings:
        property_dict = {}

        # Extract address
        address_tag = listing.find('div', class_='address')
        if address_tag:
            property_dict['Address'] = address_tag.get_text(strip=True)

        # Extract price
        price_tag = listing.find('div', class_='savePrice')
        if price_tag:
            price = price_tag.find('strong')
            if price:
                property_dict['Price'] = price.get_text(strip=True)

        # Extract details link
        details_tag = listing.find('div', class_='contViewDetailsBtn').find('a', href=True)
        if details_tag:
            property_dict['Details Link'] = details_tag['href']

        # Extract image URL
        image_tag = listing.find('div', class_='tdListingPhoto').find('img', src=True)
        if image_tag:
            property_dict['Image URL'] = 'https:' + image_tag['src']

        # Add the dictionary to the list of properties
        properties.append(property_dict)

    return properties

# Example usage
properties = extract_properties()
print(properties)


[{'Address': 'Harvestman CvAustin,TX78731', 'Price': '$1,669,988', 'Details Link': '/address/Harvestman-Cv-Austin-TX-78731/61865739_lid', 'Image URL': 'https://dlvp94zy6vayf.cloudfront.net/listingphoto/IM2TIYTDMVPWYZLQJJPUG2TSGNTWSZKSGY3XKSC2IRPUE3CHM5VVIWRQOFMWESDVOJZGG===.jpg'}, {'Address': 'Sweeney LnAustin,TX78723', 'Price': '$1,210,950', 'Details Link': '/address/Sweeney-Ln-Austin-TX-78723/61854517_lid', 'Image URL': 'https://dlvp94zy6vayf.cloudfront.net/listingphoto/OV4WEZ3NIVYXGODVJ5TXSYZQN5VTCY32MFGHKSC2IRPUE3CHM5VVIWRQOFMWESDVOJZGG===.jpg'}, {'Address': 'N Pleasant Valley RdAustin,TX78702', 'Price': '$445,500', 'Details Link': '/address/N-Pleasant-Valley-Rd-Austin-TX-78702/61854687_lid', 'Image URL': 'https://dlvp94zy6vayf.cloudfront.net/listingphoto/NJYXOYKWNZZUE2CWM5LGG6LXJI4UON3XOYYXO6TUMIWTE4LNIVAVIWRQOFMWESDVOJZGG===.jpg'}, {'Address': 'White Sands DrLakeway,TX78734', 'Price': '$526,743', 'Details Link': '/address/White-Sands-Dr-Lakeway-TX-78734/61854810_lid', 'Image URL'

In [114]:
import requests
from bs4 import BeautifulSoup
import json

# Define the base URL for relative links
base_url = 'https://mls.foreclosure.com'

# URL of the page to scrape
search_url = 'https://mls.foreclosure.com/listing/search.html?ci=austin&st=tx'

# Send an HTTP GET request to the URL
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
response = requests.get(search_url, headers=headers)

# Parse the HTML content of the page
soup = BeautifulSoup(response.text, 'html.parser')

# Look for script tags since the data is embedded in a script as JSON
scripts = soup.find_all('script', type='application/ld+json')
listings = soup.find_all('div', class_='listingRow')
print(listings)
# This will store each property's basic details
properties = []

# # Extract basic details from JSON data
# for script in scripts:
#     try:
#         data = json.loads(script.string)
#         properties.append({
#             'street_address': data.get('streetAddress'),
#             'locality': data.get('addressLocality'),
#             'region': data.get('addressRegion'),
#             'postal_code': data.get('postalCode'),
#             'latitude': data.get('geo', {}).get('latitude'),
#             'longitude': data.get('geo', {}).get('longitude'),
#             'details_url': data.get('url'),
#         })
#     except json.JSONDecodeError:
#         continue


[<div class="col-md-12 listingRow">
<div class="bLine">
<div class="tdListingPhoto fl">
<div class="conListingPhoto">
<span class="earthphoto"></span>
<a href="/address/Harvestman-Cv-Austin-TX-78731/61865739_lid" target="_top"><img alt="Harvestman-cv-Austin-TX-78731" async="" border="0" height="146" src="//dlvp94zy6vayf.cloudfront.net/listingphoto/IM2TIYTDMVPWYZLQJJPUG2TSGNTWSZKSGY3XKSC2IRPUE3CHM5VVIWRQOFMWESDVOJZGG===.jpg" width="220"/></a>
</div>
</div>
<div class="listingInfo">
<div class="conListingInfo">
<div class="messajeType">
<span class="watchHeart" title="cssbody=[tipbdy1] cssheader=[tiphdr1] header=[Save Listing] body=[You must be a member to access this feature. Try a no obligation 7-Day preview today to unlock it.] delay=[0] fade=[on] fadespeed=[0.4] offsetx=[10] offsety=[0]"><i class="icon-heart save"></i></span>
Bankruptcy
<span class="newListing">NEW</span>
</div>
<div class="contViewDetails text-right hidden-xs">
<div class="contViewDetailsBtn"><a class="btn btn-prima

In [161]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import random
import time

def slenium_login_to_site():
    """Create a browser session and log in to the website."""
    try:
        driver.get("https://mls.foreclosure.com/login.html")
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "key"))).send_keys("haiwen.zhang@gmail.com")
        driver.find_element(By.NAME, "password").send_keys("MRGSY_772")
        driver.find_element(By.ID, "btnLoginUsernamePassword").click()
        print("Login successful")
        return True
    except Exception as e:
        print("Login failed:", e)
        return False

def slenium_extract_properties():
    """Extracts basic property details from JSON embedded in script tags."""
    properties = []
    scripts = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "//script[@type='application/ld+json']"))
    )
    #     print(scripts)
    for script in scripts:
        try:
            data = json.loads(script.get_attribute('innerHTML'))
            properties.append({
                'street_address': data.get('streetAddress'),
                'locality': data.get('addressLocality'),
                'region': data.get('addressRegion'),
                'postal_code': data.get('postalCode'),
                'latitude': data.get('geo', {}).get('latitude'),
                'longitude': data.get('geo', {}).get('longitude'),
                'details_url': data.get('url'),
            })
        except json.JSONDecodeError:
            print('error')
            continue
    return properties

def slenium_fetch_details(properties):
    """Fetch and parse detailed property information using Selenium."""
    counter = 0
    for property in properties:
        if counter > 20:
            break
        details_url = property['details_url']
        if not details_url.startswith('http'):
            details_url = 'https://' + details_url
        
        driver.get(details_url)
        info_items = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li.lci'))
        )
#         info_items = driver.find_elements(By.CSS_SELECTOR, 'li.lci')
#         print(info_items, 'found')
        property_info = {}
#         print(info_items)
        for item in info_items:
            try:
                label_span = item.find_element(By.CSS_SELECTOR, 'span.attr-label')
                value_span = item.find_element(By.CSS_SELECTOR, 'span.value')
                label = label_span.text.strip().rstrip(':') if label_span else None
                value = value_span.text.strip() if value_span else 'Details not available'
                if label:
                    property_info[label] = value
            except NoSuchElementException:
                print("Expected elements not found within the item, skipping...")
                continue
        print(property_info)
        property.update(property_info)
        counter += 1
        if counter % 5 == 0:
            print(f"Scraping property No.{counter}")
        
    return properties



user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:85.0) Gecko/20100101 Firefox/85.0",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Mobile/15E148 Safari/604.1"
]

options = Options()
# options.add_argument(f"user-agent={random.choice(user_agents)}")
options.add_argument(f"user-agent={user_agents[0]}")

driver = webdriver.Chrome(options=options)

if slenium_login_to_site():
    driver.get("https://mls.foreclosure.com/listing/search.html?ci=austin&st=tx")
    properties = slenium_extract_properties()
    print('properties successfully extracted, total: ', len(properties))
    properties_updated = slenium_fetch_details(properties)
    print('properties details extracted')

#     for property in properties_updated:
#         print(property) 

    driver.quit()


Login successful
properties successfully extracted, total:  25
{'Bathrooms': '4.5', 'Living Area Size': '4,199', 'Year Built': '2002', 'Day(s) On Site': '1', 'Subdivision': 'CANYON RIM', 'Trustee Name': 'RANDOLPH N OSHEROW', 'Trustee Address': '342 W Woodlawn Ave Ste 100', 'Trustee City': 'San Antonio', 'Trustee State': 'TX', 'Trustee Zip': '78212', 'Trustee Phone': '(210) 738-3001', 'Court Name': 'District TXW, Division 1', 'Court Address': '903 San Jacinto Blvd, Suite 322', 'Baths': '4.5', 'Full Baths': '4', 'Half Baths': '1', 'No. of Stories': '2', 'Parking': 'ATTACHED GARAGE', 'Parking Sqft': '600', 'Air Conditioning': 'CENTRAL', 'Fireplaces': 'YES', 'Pool': 'CONCRETE', 'Construction Type': 'WOOD/BRICK', 'Foundation Type': 'SLAB', 'Heating Type': 'CENTRAL', 'Roof Material': 'COMPOSITION', 'Roof Type': 'HIP', 'Land Area Sqft': '10,781', 'APN': '495993', 'Property Type': 'Single Family Residence', 'Living Sqft': '4,199', 'Garage': 'ATTACHED', 'Attorney Law Firm': 'LAW OFFICE OF SUSAN

TimeoutException: Message: 
